### Project Description DataFrame

In [2]:
import os
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [3]:
data = [os.path.join('html_data', f) for f in os.listdir('html_data') if os.path.isfile(os.path.join('html_data', f))]

l = []
for d in data:
    with open(d, 'r', encoding='utf-8') as f:
        content = f.read()

    project_id = int(d.split("/")[1].split("_")[2].replace(".html", ""))

    soup = BeautifulSoup(content, 'html.parser')

    project_title = soup.find('h2', class_='heading2')
    project_description = soup.find('div', class_='ql-editor-display')    

    divs_m_botton = soup.find_all('div', class_='m-bottom')

    project_budget = None
    for div in divs_m_botton:
        title_span = div.find('span', class_='definition-data__title')
        if title_span and title_span.text.strip() == 'Budget':
            project_budget = div
            break

    project_votes = soup.find('span', class_='project-votes display-block')

    ul_project_tags = soup.find('ul', class_='tags tags--project')

    if ul_project_tags:
        li_project_tags = ul_project_tags.find_all('li')
        category = li_project_tags[0].find_all('span')
        location = li_project_tags[1].find_all('span')
    else:
        category = None
        location = None

    winner_div = soup.find('span', class_='success label project-status')

    project = {}
    
    if project_id:
        project['project_id'] = project_id

    if project_title:
        project["project_name"] = project_title.text
    
    if project_description:
        project["description"] = project_description.text

    if project_budget:
        project_budget = project_budget.text.strip().replace("Budget\n","")
        project_budget = float(project_budget.replace('€', '').replace(' ', ''))
        project['cost'] = project_budget
    
    if project_votes:
        project['votes'] = project_votes.text.replace(" votes", "")

    if category:
        project['category'] = category[1].text

    if location:
        project['district'] = location[1].text
    
    l.append(project)


In [4]:
l

[{'project_id': 132,
  'project_name': 'Lutte anti-moustiques',
  'description': "Mise en place de pièges à moustiques dans le quartier du Pont des Demoiselles: Capture des moustiques par aspiration grâce à des bornes/pièges (obturateurs) made in France simulant l'odeur humaine. Système respectueux de l'environnement.Modalités de réalisation - Avis technique :Installation de trois bornes “anti-moustiques” au sein du quartier Pont des Demoiselles. Ces bornes combinent deux facteurs : 1) La diffusion lente de CO² 2) Les composés organiques volatiles. En plus de leur efficacité, ces bornes ont également l'avantage de respecter la biodiversité. Elles seront placées aux abords des équipements publics.",
  'cost': 7500.0,
  'votes': '258',
  'category': 'Nature en ville',
  'district': '12 - Pont des Demoiselles / Ormeau / Montaudran / La Terrasse / Malepère'},
 {'project_id': 78,
  'project_name': 'Réhabiliter l’impasse des Minimes ',
  'description': 'L’Impasse Des Minimes située entre le 

##### Non-Voted Projects (1)

In [5]:
from utils import *
project_id = 3324
file = f'html_data/project_id_{project_id}.html'

p = get_project_data(file)
p['project_name'] = 'Abribus chemin de Lanusse'
p['votes'] = 0
p['category'] = 'Cadre de vie'
p['district'] = '9 - Trois Cocus / Borderouge / Croix-Daurade / Paléficat / Grand Selve'
p['cost'] = 10000

l.append(p)

ModuleNotFoundError: No module named 'utils'

In [6]:
project = pd.DataFrame(l)
project = project[~project.project_name.isna()]
project.votes = project.votes.astype('int')
project

,project_id,project_name,description,cost,votes,category,district
0,132,Lutte anti-moustiques,Mise en place de pièges à moustiques dans le q...,7500.0,258,Nature en ville,12 - Pont des Demoiselles / Ormeau / Montaudra...
1,78,Réhabiliter l’impasse des Minimes,L’Impasse Des Minimes située entre le boulevar...,150000.0,23,Cadre de vie,8 - Minimes / Barrière de Paris / Ponts-Jumeau...
2,122,Aire convivialité espace vert rue de l'Isére /...,Création d'une aire de convivialité dans l'esp...,75000.0,5,Cadre de vie,11 - Bonhoure / Guilheméry / Château de l'Hers...
3,149,Un jardin en mouvement le long de la voie ferr...,Pour accroitre la connaissance des citadins da...,11000.0,50,Nature en ville,5 - Saint-Michel / Saint-Agne / Empalot / Le B...
4,129,Zone de Street Workout - Parc de la boisseraie,Pour favoriser la pratique sportive et l'entre...,20000.0,31,Cadre de vie,12 - Pont des Demoiselles / Ormeau / Montaudra...
...,...,...,...,...,...,...,...
203,98,Equipements sportifs,"Je souhaite des équipements sportifs de type ""...",15000.0,26,Cadre de vie,9 - Trois Cocus / Borderouge / Croix-Daurade /...
204,88,Remplacer les feux tricolores actuels de la Ro...,Le feu récompense est un moyen simple pour inc...,55000.0,9,Cadre de vie,7 - Sept Deniers / Ginestous-Sesquières / Lalande
205,109,Création d'une Venelle quartier Soupetard Ros...,Il n'est à ce jour prévu aucune communication ...,100000.0,36,Cadre de vie,10 - Jolimont / Soupetard / Roseraie / Gloire ...
206,131,Le développement de l'auto partage,"Bonjour, Afin de pouvoir poursuivre réduire la...",4500.0,35,Éco-mobilité,12 - Pont des Demoiselles / Ormeau / Montaudra...


In [ ]:
## Districts

In [ ]:
district_str = ""
for d in project.district.tolist():
    district_str=district_str+f"{d};"

print(district_str)

### Detail Validation

In [7]:
detail = pd.read_excel('PBToulouse2022/idvotant-idprojet.xlsx')

## cleaned voter file
detail = detail[detail.vote_finished==True]
detail = detail[detail.checked_out_at < '2022-11-01']
detail.project_id = detail.project_id.astype('int')

detail

,ID,project_id,project_title,created_at,checked_out_at,project_url,vote_finished
0,56,215,CHEMIN DE BALADE AU CŒUR DE NOS QUARTIERS,2022-10-03 12:51:57.621,2022-10-03 12:53:28.056,https://jeparticipe.metropole.toulouse.fr/proc...,True
1,107,215,CHEMIN DE BALADE AU CŒUR DE NOS QUARTIERS,2022-10-03 14:44:45.279,2022-10-03 14:47:20.746,https://jeparticipe.metropole.toulouse.fr/proc...,True
2,114,215,CHEMIN DE BALADE AU CŒUR DE NOS QUARTIERS,2022-10-03 15:14:09.332,2022-10-03 15:14:51.580,https://jeparticipe.metropole.toulouse.fr/proc...,True
3,115,215,CHEMIN DE BALADE AU CŒUR DE NOS QUARTIERS,2022-10-03 15:18:12.251,2022-10-03 15:22:43.887,https://jeparticipe.metropole.toulouse.fr/proc...,True
4,132,215,CHEMIN DE BALADE AU CŒUR DE NOS QUARTIERS,2022-10-03 17:28:34.272,2022-10-03 17:28:46.168,https://jeparticipe.metropole.toulouse.fr/proc...,True
...,...,...,...,...,...,...,...
12480,5002,72,Anti-moustiques - Distribuer des nichoirs à ch...,2022-10-30 17:18:45.205,2022-10-30 17:20:36.086,https://jeparticipe.metropole.toulouse.fr/proc...,True
12481,5020,72,Anti-moustiques - Distribuer des nichoirs à ch...,2022-10-30 19:18:23.254,2022-10-30 19:20:02.884,https://jeparticipe.metropole.toulouse.fr/proc...,True
12482,5027,72,Anti-moustiques - Distribuer des nichoirs à ch...,2022-10-30 19:56:13.973,2022-10-30 20:08:35.799,https://jeparticipe.metropole.toulouse.fr/proc...,True
12483,5097,72,Anti-moustiques - Distribuer des nichoirs à ch...,2022-10-31 11:00:27.279,2022-10-31 11:02:23.781,https://jeparticipe.metropole.toulouse.fr/proc...,True


In [15]:
detail.to_csv("projectDetails.csv", sep=";", index=False)

In [8]:
votes_by_projects = detail.groupby(['project_id']).ID.nunique()
votes_by_projects = votes_by_projects.reset_index().rename(columns={'ID': 'votes'}, inplace=False)
votes_by_projects

,project_id,votes
0,4,36
1,5,358
2,6,61
3,7,467
4,8,49
...,...,...
202,209,29
203,210,49
204,211,49
205,214,77


In [9]:
val_df = pd.merge(left=project, right=votes_by_projects, on='project_id', how='left')
print("missmatch in votes : ", len(val_df[val_df.votes_x != val_df.votes_y]))

missmatch in votes :  0


In [11]:
finalAgg_ = project.filter(['project_id', 'project_name', 'description', 'category', 'cost','district', 'votes'])
finalAgg_.to_csv("projectsAgg2.csv", sep=";", index=False)

In [12]:
# cleaned detail voting df:
detail = detail[detail.project_id.isin(project.project_id.tolist())]

In [13]:
#set of voters
voters = set(detail.ID.tolist())
print("voter_id;project_id")
for v in voters:
    votesdf = detail[detail.ID == v]
    print(f"{v};{votesdf.project_id.tolist()}")

voter_id;project_id
9;[115]
11;[115]
12;[35, 36, 34]
13;[81, 88, 5]
15;[66, 34, 68]
16;[136, 134, 115]
17;[58, 52, 54]
18;[156, 151]
19;[136, 134, 115]
20;[68]
21;[73, 79, 77]
22;[6, 19, 5]
23;[68]
24;[163]
25;[68, 7, 94]
26;[71]
27;[95]
28;[115]
29;[115]
30;[82, 91, 94]
32;[157, 148, 139]
35;[136, 132, 128]
36;[143, 136, 132]
37;[173, 185, 197]
38;[144]
39;[27, 29, 30]
40;[115]
41;[136, 132, 128]
42;[143, 144, 139]
43;[115]
44;[82, 90, 84]
45;[132, 128, 137]
46;[6, 13, 162]
47;[115]
48;[68]
49;[67, 66, 64]
50;[136, 132, 128]
51;[144, 26, 137]
52;[44, 41, 45]
53;[67, 66, 68]
54;[143, 144, 153]
58;[144, 141, 139]
59;[148, 150, 155]
61;[143, 144, 146]
62;[99, 95, 91]
63;[136, 129, 135]
64;[136, 125, 133]
65;[157, 150, 153]
66;[6, 128, 137]
68;[136, 132, 137]
70;[139]
71;[63, 68, 69]
72;[71, 91, 74]
73;[115]
74;[136, 203, 7]
76;[62, 66, 70]
77;[136, 156, 151]
78;[201]
79;[58]
80;[44, 92, 5]
81;[136, 134]
83;[136]
86;[124]
87;[157, 154, 149]
89;[115]
90;[157, 153, 149]
91;[11, 172, 79]
92;

In [14]:
len(voters)

4532